In [2]:
# Makes print and division act like Python 3
from __future__ import print_function, division

# Import the usual libraries
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

# Enable inline plotting at lower left
%matplotlib inline

from IPython.display import display, Latex, clear_output
from matplotlib.backends.backend_pdf import PdfPages

In [3]:
import pynrc
from pynrc import nrc_utils
from pynrc.nrc_utils import S
from pynrc.obs_nircam import model_to_hdulist
pynrc.setup_logging('WARNING', verbose=False)

In [5]:
bp_k = nrc_utils.bp_2mass('k')

In [4]:
#                source,   sptype, kmag
args_sources = [('HD19467', 'G3V', 5.4),
                ('HD19096', 'K0V', 5.3)]

In [ ]:
nrc = pynrc.NIRCam(filter='F335M', pupil='CIRCLYOT', ND_acq=True,
                   wind_mode='WINDOW', xpix=64, ypix=64)

## ND Target Acquisition Times

In [30]:
for i in range(len(args_sources)):

    name_sci, spt_sci, kmag_sci = args_sources[i]

    # Science target spectrum
    sp = pynrc.stellar_spectrum(spt_sci, kmag_sci, 'vegamag', bp_k, catname='ck04models')
    sp.name = name_sci

    # Cycle through allowed TA ngroups
    ng_levels = [3,5,9,17,33,65]
    print(sp.name)
    for ng in ng_levels:
        pynrc.setup_logging('ERROR', verbose=False)
        tbl = nrc.ramp_optimize(sp, snr_goal=50, snr_frac=0.01, well_frac_max=0.5, verbose=False,
                                ng_min=ng, ng_max=ng, nint_max=1, patterns=['RAPID', 'BRIGHT1'])
        nrow = len(tbl)
        if nrow>0:
            print(tbl)
            break
    print('')

pynrc.setup_logging('WARNING', verbose=False)

HD19467
 Pattern   NGRP NINT   t_int     t_exp     t_acq     SNR      Well     eff   
---------- ---- ---- --------- --------- --------- -------- -------- --------
BRIGHT1      17    1      1.66      1.66      1.71     54.1    0.010   41.370

HD19096
 Pattern   NGRP NINT   t_int     t_exp     t_acq     SNR      Well     eff   
---------- ---- ---- --------- --------- --------- -------- -------- --------
BRIGHT1      17    1      1.66      1.66      1.71     58.0    0.011   44.325



## Astrometric Confirmation Settings

Want to be sensitive to objects with brightnesses of 10-18 mags at 3.3um

In [6]:
nrc = pynrc.NIRCam(filter='F335M', pupil='CIRCLYOT', wind_mode='FULL')

In [7]:
sp_dim = pynrc.stellar_spectrum('flat', 17, 'vegamag', nrc.bandpass)
sp_bright = pynrc.stellar_spectrum('flat', 11, 'vegamag', nrc.bandpass)
nrc = pynrc.NIRCam(filter='F335M', pupil='CIRCLYOT', wind_mode='FULL')
tbl = nrc.ramp_optimize(sp_dim, sp_bright, snr_goal=30, snr_frac=0.1, ng_min=2, ng_max=10, nint_min=3,
                        well_frac_max=1.2, patterns=['RAPID', 'BRIGHT1', 'SHALLOW2'], verbose=True)


BRIGHT1
RAPID
SHALLOW2
 Pattern   NGRP NINT   t_int     t_exp     t_acq     SNR      Well     eff   
---------- ---- ---- --------- --------- --------- -------- -------- --------
BRIGHT1       3    3     53.68    161.05    193.27     31.5    1.040    2.263
BRIGHT1       3    4     53.68    214.74    257.69     36.3    1.040    2.263
RAPID         4    4     42.95    171.79    214.74     29.3    0.832    2.001
RAPID         4    5     42.95    214.74    268.42     32.8    0.832    2.001
RAPID         4    6     42.95    257.68    322.11     35.9    0.832    2.001
